# L10a: The GloVe Model for Word Embeddings
In this lecture, we explore the GloVe (Global Vectors for Word Representation) model, a popular method for learning word embeddings by leveraging global word co-occurrence statistics from a corpus.

> __Learning Objectives:__
> 
> Three learning objectives go here.

Let's get started!
___

## Summary
One concise, direct summary sentence goes here.

> __Key Takeaways:__
> 
> Three key takeaways go here.

One concise, direct summary sentence goes here.
___